In [ ]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import os

In [ ]:
# Set window size for moving average (e.g., average over 100 episodes)
WINDOW_SIZE = 200

In [ ]:
jump_training_csv = "csv_data/dqn_jump.csv"
vanilla_training_csv = "csv_data/dqn.csv"

jump_data_folder = "csv_data/jump_test_data"
vanilla_data_folder = "csv_data/vanilla_test_data"

In [ ]:
jump_train_df = pd.read_csv(jump_training_csv)
vanilla_train_df = pd.read_csv(vanilla_training_csv)

In [ ]:
jump_train_df["wind_vector"] = jump_train_df["wind_vector"].apply(ast.literal_eval)
vanilla_train_df["wind_vector"] = vanilla_train_df["wind_vector"].apply(ast.literal_eval)

jump_train_df["wind_vector"] = jump_train_df["wind_vector"].apply(lambda x: [round(val, 2) for val in x])
jump_train_df["height"] = jump_train_df["height"].apply(lambda x: round(x, 2))
jump_train_df["reward"] = jump_train_df["reward"].apply(lambda x: round(x, 2))

vanilla_train_df["wind_vector"] = vanilla_train_df["wind_vector"].apply(lambda x: [round(val, 2) for val in x])
vanilla_train_df["height"] = vanilla_train_df["height"].apply(lambda x: round(x, 2))
vanilla_train_df["reward"] = vanilla_train_df["reward"].apply(lambda x: round(x, 2))

In [ ]:
jump_train_df.head()

In [ ]:
# Compute moving average of rewards
jump_train_df['reward_ma'] = jump_train_df['reward'].rolling(window=WINDOW_SIZE, min_periods=1).mean()

plt.figure(figsize=(14, 7))

# Create primary axis for reward
ax1 = plt.gca()  # Get current axis
ax1.plot(jump_train_df['episode'], jump_train_df['reward_ma'], 
         linestyle='-', color='g', linewidth=2, label='Reward (MA)')
ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Moving Avg. Reward', fontsize=12, color='g')
ax1.tick_params(axis='y', labelcolor='g')
ax1.grid(True)

# Create secondary axis for epsilon (in red)
ax2 = ax1.twinx()
ax2.plot(jump_train_df['episode'], jump_train_df['epsilon'], 
         linestyle='--', color='r', alpha=0.7, linewidth=1.5, label='Epsilon')
ax2.set_ylabel('Epsilon (Exploration)', fontsize=12, color='r')
ax2.tick_params(axis='y', labelcolor='r')
ax2.set_ylim(0, 1.05)  # Assuming epsilon is between 0 and 1

# Title and legend
plt.title(f'FILTERED | Reward (Moving Avg.) & Epsilon Decay (Window={WINDOW_SIZE})', fontsize=16, pad=20)

# Combine legends from both axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Adjust x-axis ticks
max_episode = jump_train_df['episode'].max()
plt.xticks(range(0, max_episode + 1, max(1, max_episode // 10)))

plt.tight_layout()
plt.show()

In [ ]:
# Compute moving average of success rate
smoothed_success = jump_train_df['success'].rolling(window=WINDOW_SIZE).mean()

plt.figure(figsize=(14, 7))

# Create primary axis for success rate (green)
ax1 = plt.gca()
ax1.plot(jump_train_df['episode'], smoothed_success, 
         linestyle='-', color='g', linewidth=2, label='Success Rate (MA)')
ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Success Rate', fontsize=12, color='g')
ax1.tick_params(axis='y', labelcolor='g')
ax1.grid(True)
ax1.set_ylim(0, 1.0)
ax1.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax1.set_yticklabels(['0%', '20%', '40%', '60%', '80%', '100%'])

# Create secondary axis for epsilon (red)
ax2 = ax1.twinx()
ax2.plot(jump_train_df['episode'], jump_train_df['epsilon'], 
         linestyle='--', color='r', alpha=0.7, linewidth=1.5, label='Epsilon')
ax2.set_ylabel('Epsilon (Exploration)', fontsize=12, color='r')
ax2.tick_params(axis='y', labelcolor='r')
ax2.set_ylim(0, 1.05)  # Assuming epsilon ranges [0,1]

# Title and combined legend
plt.title(f'FILTERED | Success Rate & Epsilon Decay (Window={WINDOW_SIZE})', fontsize=16, pad=20)
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Adjust x-axis ticks
max_episode = jump_train_df['episode'].max()
plt.xticks(range(0, max_episode + 1, max(1, max_episode // 10)))

plt.tight_layout()
plt.show()

In [ ]:
# Compute moving average of rewards
vanilla_train_df['reward_ma'] = vanilla_train_df['reward'].rolling(window=WINDOW_SIZE, min_periods=1).mean()

plt.figure(figsize=(14, 7))

# Create primary axis for reward
ax1 = plt.gca()  # Get current axis
ax1.plot(vanilla_train_df['episode'], vanilla_train_df['reward_ma'], 
         linestyle='-', color='b', linewidth=2, label='Reward (MA)')
ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Moving Avg. Reward', fontsize=12, color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(True)

# Create secondary axis for epsilon (in red)
ax2 = ax1.twinx()
ax2.plot(vanilla_train_df['episode'], vanilla_train_df['epsilon'], 
         linestyle='--', color='r', alpha=0.7, linewidth=1.5, label='Epsilon')
ax2.set_ylabel('Epsilon (Exploration)', fontsize=12, color='r')
ax2.tick_params(axis='y', labelcolor='r')
ax2.set_ylim(0, 1.05)  # Assuming epsilon is between 0 and 1

# Title and legend
plt.title(f'Vanilla | Reward (Moving Avg.) & Epsilon Decay (Window={WINDOW_SIZE})', fontsize=16, pad=20)

# Combine legends from both axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Adjust x-axis ticks
max_episode = vanilla_train_df['episode'].max()
plt.xticks(range(0, max_episode + 1, max(1, max_episode // 10)))

plt.tight_layout()
plt.show()

In [ ]:
# Compute moving average of success rate
smoothed_success = vanilla_train_df['success'].rolling(window=WINDOW_SIZE).mean()

plt.figure(figsize=(14, 7))

# Create primary axis for success rate (green)
ax1 = plt.gca()
ax1.plot(vanilla_train_df['episode'], smoothed_success, 
         linestyle='-', color='b', linewidth=2, label='Success Rate (MA)')
ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Success Rate', fontsize=12, color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(True)
ax1.set_ylim(0, 1.0)
ax1.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax1.set_yticklabels(['0%', '20%', '40%', '60%', '80%', '100%'])

# Create secondary axis for epsilon (red)
ax2 = ax1.twinx()
ax2.plot(vanilla_train_df['episode'], vanilla_train_df['epsilon'], 
         linestyle='--', color='r', alpha=0.7, linewidth=1.5, label='Epsilon')
ax2.set_ylabel('Epsilon (Exploration)', fontsize=12, color='r')
ax2.tick_params(axis='y', labelcolor='r')
ax2.set_ylim(0, 1.05)  # Assuming epsilon ranges [0,1]

# Title and combined legend
plt.title(f'Vanilla | Success Rate & Epsilon Decay (Window={WINDOW_SIZE})', fontsize=16, pad=20)
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Adjust x-axis ticks
max_episode = vanilla_train_df['episode'].max()
plt.xticks(range(0, max_episode + 1, max(1, max_episode // 10)))

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))

# Dataset 1 - Solid blue line
jump_train_df['steps_ma'] = jump_train_df['steps'].rolling(window=WINDOW_SIZE).mean()
plt.plot(jump_train_df['episode'], jump_train_df['steps_ma'], 
         color='g', linewidth=2, label='Filtered Buffer')

# Dataset 2 - Dashed orange line
vanilla_train_df['steps_ma'] = vanilla_train_df['steps'].rolling(window=WINDOW_SIZE).mean()
plt.plot(vanilla_train_df['episode'], vanilla_train_df['steps_ma'], 
         color='b', linestyle='--', linewidth=2, label='Vanilla')

plt.title(f'Step Count Comparison (Moving Average, Window={WINDOW_SIZE})', fontsize=16)
plt.xlabel('Episode', fontsize=12)
plt.ylabel('Steps (Moving Average)', fontsize=12)
plt.grid(True, alpha=0.3)

# Auto-adjust x-axis
max_episode = max(jump_train_df['episode'].max(), vanilla_train_df['episode'].max())
plt.xticks(range(0, max_episode + 1, max(1, max_episode // 10)))

# Add legend and tight layout
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))

# Calculate total actions per episode for both datasets
jump_train_df['total_actions'] = (jump_train_df['actions_forward'] + 
                                 jump_train_df['actions_backward'] + 
                                 jump_train_df['actions_left'] + 
                                 jump_train_df['actions_right'])

vanilla_train_df['total_actions'] = (vanilla_train_df['actions_forward'] + 
                                    vanilla_train_df['actions_backward'] + 
                                    vanilla_train_df['actions_left'] + 
                                    vanilla_train_df['actions_right'])

# Compute moving averages
jump_train_df['actions_ma'] = jump_train_df['total_actions'].rolling(WINDOW_SIZE).mean()
vanilla_train_df['actions_ma'] = vanilla_train_df['total_actions'].rolling(WINDOW_SIZE).mean()

# Plot both datasets
plt.plot(jump_train_df['episode'], jump_train_df['actions_ma'], 
         color='g', linewidth=2.5, label='Filtered Buffer (MA)')
plt.plot(vanilla_train_df['episode'], vanilla_train_df['actions_ma'], 
         color='b', linestyle='--', linewidth=2.5, label='Vanilla (MA)')

# Style enhancements
plt.title('Total Actions Comparison (200-Episode Moving Average)', fontsize=18, pad=20)
plt.xlabel('Episode', fontsize=14)
plt.ylabel('Average Total Actions', fontsize=14)
plt.grid(True, alpha=0.2)

# Dynamic x-axis ticks
max_episode = max(jump_train_df['episode'].max(), vanilla_train_df['episode'].max())
plt.xticks(range(0, max_episode + 1, max(1, max_episode // 10)), fontsize=12)
plt.yticks(fontsize=12)

# Legend with shadow
legend = plt.legend(fontsize=12, framealpha=1, shadow=True)
legend.get_frame().set_edgecolor('black')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))

# --- Terminal Distance (Left Axis) ---
ax1 = plt.gca()

# Filtered Buffer - Terminal Distance
jump_train_df['terminal_dist_ma'] = jump_train_df['terminal_distance'].rolling(WINDOW_SIZE).mean()
ax1.plot(jump_train_df['episode'], jump_train_df['terminal_dist_ma'], 
         color='g', linewidth=2.5, label='Filtered Buffer (Distance)')

# Vanilla - Terminal Distance
vanilla_train_df['terminal_dist_ma'] = vanilla_train_df['terminal_distance'].rolling(WINDOW_SIZE).mean()
ax1.plot(vanilla_train_df['episode'], vanilla_train_df['terminal_dist_ma'], 
         color='b', linestyle='--', linewidth=2.5, label='Vanilla (Distance)')

ax1.set_xlabel('Episode', fontsize=14)
ax1.set_ylabel('Terminal Distance (MA)', fontsize=14, color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.grid(True, alpha=0.2)

# --- Epsilon (Right Axis) ---
ax2 = ax1.twinx()

# Filtered Buffer - Epsilon
ax2.plot(jump_train_df['episode'], jump_train_df['epsilon'], 
         color='r', alpha=0.7, linewidth=1.5, label='Epsilon (ε)')

ax2.set_ylabel('Epsilon (ε)', fontsize=14)
ax2.set_ylim(0, 1.05)  # Epsilon range
ax2.tick_params(axis='y')

# --- Styling ---
plt.title(f'Terminal Distance (MA) and Epsilon Comparison\n(Window={WINDOW_SIZE} episodes)', 
          fontsize=16, pad=20)

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=12)

# Dynamic x-axis
max_episode = max(jump_train_df['episode'].max(), vanilla_train_df['episode'].max())
plt.xticks(range(0, max_episode + 1, max(1, max_episode // 10)))

plt.tight_layout()
plt.show()

In [ ]:
dataframes = {}
data_dir = "csv_data/jump_test_data/"

for filename in sorted(os.listdir(data_dir)):
    filepath = os.path.join(data_dir, filename)
    df = pd.read_csv(filepath)
    df["wind_vector"] = df["wind_vector"].apply(ast.literal_eval)
    df["wind_vector"] = df["wind_vector"].apply(lambda x: [round(val, 2) for val in x])
    df["height"] = df["height"].apply(lambda x: round(x, 2))
    df["reward"] = df["reward"].apply(lambda x: round(x, 2))
    df["wind"] = df["wind_vector"].apply(lambda x: np.linalg.norm(x))
    key = filename.split(".")[0][-3:]
    dataframes[key] = df
dataframes.keys()

In [ ]:
dataframes["100"].head()

In [ ]:
# Heights to analyze (in meters)
heights = [50, 100, 150, 200, 250, 300, 350, 400]

# Calculate success rates for each model at each height
success_rates = {}
for model_name, df in dataframes.items():
    rates = []
    for h in heights:
        # Include ±50 meter range
        subset = df[(df['height'] >= h-50) & (df['height'] <= h+50)]
        success_rate = subset['success'].mean() if len(subset) > 0 else 0
        rates.append(success_rate)
    success_rates[model_name] = rates

# Plot setup
bar_width = 0.15
index = np.arange(len(heights))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

plt.figure(figsize=(15, 8))

# Plot bars for each model
for i, (model_name, rates) in enumerate(success_rates.items()):
    plt.bar(index + i*bar_width, rates, bar_width, 
            color=colors[i], label=f"{model_name} (±50m)")

# Customize plot
plt.xlabel('Base Height (meters)', fontsize=12)
plt.ylabel('Success Rate', fontsize=12)
plt.title('Success Rate by Height Across Models', fontsize=16)
plt.xticks(index + bar_width*(len(dataframes)-1)/2, 
           [f"{h}m" for h in heights], fontsize=10)
plt.yticks(np.arange(0, 1.1, 0.1), 
           [f'{int(x*100)}%' for x in np.arange(0, 1.1, 0.1)])
plt.ylim(0, 1)
plt.grid(axis='y', alpha=0.3)

# Add legend and adjust layout
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
           title="Models trained on:")
plt.tight_layout()

plt.show()

In [ ]:
# Defined heights
heights = [50, 100, 150, 200, 250, 300, 350, 400]
bar_width = 0.35
index = np.arange(len(heights))

# Create one plot per model
for model_name, df in dataframes.items():
    plt.figure(figsize=(12, 6))
    
    # Prepare data storage
    success_dists = []
    failed_dists = []
    success_rates = []
    
    for h in heights:
        # Filter by height
        height_data = df[df['height'] == h]
        
        if len(height_data) > 0:
            # Successful landings
            success_data = height_data[height_data['success'] == 1]
            success_dist = success_data['terminal_distance'].mean() if not success_data.empty else np.nan
            success_dists.append(success_dist)
            
            # Failed landings
            failed_data = height_data[height_data['success'] == 0]
            failed_dist = failed_data['terminal_distance'].mean() if not failed_data.empty else np.nan
            failed_dists.append(failed_dist)
            
            # Success rate
            success_rate = len(success_data)/len(height_data)
            success_rates.append(success_rate)
        else:
            success_dists.append(np.nan)
            failed_dists.append(np.nan)
            success_rates.append(np.nan)
    
    # Create bars for terminal distances
    bars1 = plt.bar(index - bar_width/2, success_dists, bar_width,
                   color='green', alpha=0.7, label='Successful Landings')
    bars2 = plt.bar(index + bar_width/2, failed_dists, bar_width,
                   color='red', alpha=0.7, label='Failed Landings')
    
    # Add value labels
    for bar, dist in zip(bars1, success_dists):
        if not np.isnan(dist):
            plt.text(bar.get_x() + bar.get_width()/2, dist + 0.5,
                    f'{dist:.1f}m', ha='center', color='green')
    
    for bar, dist in zip(bars2, failed_dists):
        if not np.isnan(dist):
            plt.text(bar.get_x() + bar.get_width()/2, dist + 0.5,
                    f'{dist:.1f}m', ha='center', color='red')
    
    # Create twin axis for success rate
    ax2 = plt.gca().twinx()
    ax2.plot(index, [r*100 for r in success_rates], 
             'b-o', linewidth=2, markersize=8, label='Success Rate')
    
    # Customize plot
    plt.title(f'{model_name} Landing Performance by Height', pad=20)
    plt.xlabel('Height (m)')
    plt.gca().set_ylabel('Terminal Distance (m)')
    plt.gca().set_xticks(index)
    plt.gca().set_xticklabels([f"{h}m" for h in heights])
    ax2.set_ylabel('Success Rate (%)')
    ax2.set_ylim(0, 110)
    
    # Combine legends
    lines, labels = plt.gca().get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    plt.legend(lines + lines2, labels + labels2, loc='upper right')
    
    plt.grid(axis='y', alpha=0.2)
    plt.tight_layout()
    plt.show()

In [171]:
# Define the metrics and columns we're interested in
metrics = ['mean', 'std', 'count']
columns_of_interest = ['terminal_distance', 'reward', 'steps', 'success', 'wind']

# Process each dataframe
for model_name, df in dataframes.items():
    print(f"\n{'='*50}")
    print(f"MODEL: {model_name.upper()}")
    print(f"{'='*50}")
    
    # Group by height and calculate statistics
    grouped = df.groupby('height')[columns_of_interest].agg(metrics)
    
    # Print statistics for successful landings
    success_df = df[df['success'] == 1]
    if not success_df.empty:
        success_grouped = success_df.groupby('height')[columns_of_interest].agg(metrics)
        print("\nSUCCESSFUL LANDINGS:")
        print(success_grouped)
    
    # Print statistics for failed landings
    failed_df = df[df['success'] == 0]
    if not failed_df.empty:
        failed_grouped = failed_df.groupby('height')[columns_of_interest].agg(metrics)
        print("\nFAILED LANDINGS:")
        print(failed_grouped)
    
    # Print overall statistics
    print("\nOVERALL STATISTICS:")
    print(grouped)


MODEL: 100

SUCCESSFUL LANDINGS:
       terminal_distance                     reward                  \
                    mean       std count       mean       std count   
height                                                                
50              2.247444  0.808944   231  13.011991  4.946887   231   
100             2.391617  0.913756   233  12.813391  5.694228   233   
150             2.606490  0.979051   226  10.146770  6.682509   226   
200             3.417382  1.067784   110   4.996364  5.892322   110   
250             3.342966  1.188806    15   2.234667  4.464815    15   
300             3.812435  0.832851    15   1.123333  4.169321    15   
350             4.263177  0.585152     9  -0.008889  2.524373     9   
400             3.957014  1.196977     4  -1.422500  2.620921     4   

             steps                   success                  wind            \
              mean         std count    mean  std count       mean       std   
height                  

In [ ]:
dataframes = {}
data_dir = "csv_data/vanilla_test_data/"

for filename in sorted(os.listdir(data_dir)):
    filepath = os.path.join(data_dir, filename)
    df = pd.read_csv(filepath)
    df["wind_vector"] = df["wind_vector"].apply(ast.literal_eval)
    df["wind_vector"] = df["wind_vector"].apply(lambda x: [round(val, 2) for val in x])
    df["height"] = df["height"].apply(lambda x: round(x, 2))
    df["reward"] = df["reward"].apply(lambda x: round(x, 2))
    key = filename.split(".")[0][-3:]
    dataframes[key] = df
dataframes.keys()

In [ ]:
# Heights to analyze (in meters)
heights = [50, 100, 150, 200, 250, 300, 350, 400]

# Calculate success rates for each model at each height
success_rates = {}
for model_name, df in dataframes.items():
    rates = []
    for h in heights:
        # Include ±50 meter range
        subset = df[(df['height'] >= h-50) & (df['height'] <= h+50)]
        success_rate = subset['success'].mean() if len(subset) > 0 else 0
        rates.append(success_rate)
    success_rates[model_name] = rates

# Plot setup
bar_width = 0.15
index = np.arange(len(heights))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

plt.figure(figsize=(15, 8))

# Plot bars for each model
for i, (model_name, rates) in enumerate(success_rates.items()):
    plt.bar(index + i*bar_width, rates, bar_width, 
            color=colors[i], label=f"{model_name} (±50m)")

# Customize plot
plt.xlabel('Base Height (meters)', fontsize=12)
plt.ylabel('Success Rate', fontsize=12)
plt.title('Success Rate by Height Across Models', fontsize=16)
plt.xticks(index + bar_width*(len(dataframes)-1)/2, 
           [f"{h}m" for h in heights], fontsize=10)
plt.yticks(np.arange(0, 1.1, 0.1), 
           [f'{int(x*100)}%' for x in np.arange(0, 1.1, 0.1)])
plt.ylim(0, 1)
plt.grid(axis='y', alpha=0.3)

# Add legend and adjust layout
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
           title="Models trained on:")
plt.tight_layout()

plt.show()

In [172]:
dataframes = {}
data_dir = "csv_data/vanilla_test_data/"

for filename in sorted(os.listdir(data_dir)):
    filepath = os.path.join(data_dir, filename)
    df = pd.read_csv(filepath)
    df["wind_vector"] = df["wind_vector"].apply(ast.literal_eval)
    df["wind_vector"] = df["wind_vector"].apply(lambda x: [round(val, 2) for val in x])
    df["height"] = df["height"].apply(lambda x: round(x, 2))
    df["reward"] = df["reward"].apply(lambda x: round(x, 2))
    df["wind"] = df["wind_vector"].apply(lambda x: np.linalg.norm(x))
    key = filename.split(".")[0][-3:]
    dataframes[key] = df
dataframes.keys()

dict_keys(['100', '150', '200', '250', '300', '350'])

In [173]:
# Define the metrics and columns we're interested in
metrics = ['mean', 'std', 'count']
columns_of_interest = ['terminal_distance', 'reward', 'steps', 'success', 'wind']

# Process each dataframe
for model_name, df in dataframes.items():
    print(f"\n{'='*50}")
    print(f"MODEL: {model_name.upper()}")
    print(f"{'='*50}")
    
    # Group by height and calculate statistics
    grouped = df.groupby('height')[columns_of_interest].agg(metrics)
    
    # Print statistics for successful landings
    success_df = df[df['success'] == 1]
    if not success_df.empty:
        success_grouped = success_df.groupby('height')[columns_of_interest].agg(metrics)
        print("\nSUCCESSFUL LANDINGS:")
        print(success_grouped)
    
    # Print statistics for failed landings
    failed_df = df[df['success'] == 0]
    if not failed_df.empty:
        failed_grouped = failed_df.groupby('height')[columns_of_interest].agg(metrics)
        print("\nFAILED LANDINGS:")
        print(failed_grouped)
    
    # Print overall statistics
    print("\nOVERALL STATISTICS:")
    print(grouped)


MODEL: 100

FAILED LANDINGS:
       terminal_distance                       reward                  \
                    mean          std count      mean       std count   
height                                                                  
50            124.182276    25.952865   250  -7.98612  5.227689   250   
100            46.809306    16.001826   250  -7.94152  5.421007   250   
150          2177.776766  2115.482650   250 -12.92460  9.558331   250   
200          1774.916607  1766.654724   250 -12.21576  8.411109   250   
250          1581.181032  1675.196022   250 -11.76524  8.262652   250   
300          1658.378722  1751.715908   250 -12.57632  8.742764   250   
350          1657.619626  1819.536715   250 -12.87960  9.279066   250   
400          1715.187148  1664.091933   250 -13.49212  8.640539   250   

          steps                   success                  wind            \
           mean         std count    mean  std count       mean       std   
height      